In [5]:
import pandas as pd

df = pd.read_csv('vggsound.csv', header=None)
df.head()
column_names = ['video_id', 'start_time', 'action', 'split', 'idx']
df.columns = column_names

df['end_time'] = df['start_time'] + 10

columns_names = ['video_id', 'start_time', 'end_time', 'action', 'split', 'idx']

df = df[columns_names]

In [6]:
# Split the dataframe into 6 different dataframes  with near equal number of rows

df1 = df.iloc[:20000]
df2 = df.iloc[20000:40000]
df3 = df.iloc[40000:60000]
df4 = df.iloc[60000:80000]
df5 = df.iloc[80000:100000]
df6 = df.iloc[100000:120000]
df7 = df.iloc[120000:140000]
df8 = df.iloc[140000:160000]
df9 = df.iloc[160000:180000]
df10 = df.iloc[180000:]




In [7]:
# Save the dataframes to csv files with filename as vgg_sound_1.csv, vgg_sound_2.csv, vgg_sound_3.csv, vgg_sound_4.csv, vgg_sound_5.csv, vgg_sound_6.csv, vgg_sound_7.csv, vgg_sound_8.csv, vgg_sound_9.csv, vgg_sound_10.csv   

df1.to_csv('vgg_sound_1.csv', index=False)
df2.to_csv('vgg_sound_2.csv', index=False)
df3.to_csv('vgg_sound_3.csv', index=False)
df4.to_csv('vgg_sound_4.csv', index=False)
df5.to_csv('vgg_sound_5.csv', index=False)
df6.to_csv('vgg_sound_6.csv', index=False)
df7.to_csv('vgg_sound_7.csv', index=False)
df8.to_csv('vgg_sound_8.csv', index=False)
df9.to_csv('vgg_sound_9.csv', index=False)
df10.to_csv('vgg_sound_10.csv', index=False)


In [2]:
import os
import csv
import yt_dlp
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
# Set paths for storing downloaded data
DATA_DIR = "/research/iprobe-dardaniy/VGGSoundData"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)
# Path to the VGGSound CSV file
CSV_FILE = "vggsound.csv"
# Function to download video and extract audio
def process_video(video_info):
    video_id, start_sec, label, split = video_info
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    output_dir = TRAIN_DIR if split.lower() == "train" else TEST_DIR
    video_path = os.path.join(output_dir, f"{video_id}.mp4")
    audio_path = os.path.join(output_dir, f"{video_id}.wav")
    metadata_path = os.path.join(output_dir, f"{video_id}.txt")
    # Skip if already processed
    if os.path.exists(audio_path):
        return f"Skipping {video_id} (already processed)"
    # Download video using yt-dlp
    ydl_opts = {
    "format": "bestvideo+bestaudio/best",
    "outtmpl": video_path,
    "quiet": True,
    "postprocessors": [{"key": "FFmpegVideoConvertor", "preferedformat": "mp4"}],
    "cookies": "www.youtube.com_cookies.txt",  # Add your exported cookies file
}
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
    except Exception as e:
        return f"Failed to download {video_id}: {e}"
    # Extract audio using ffmpeg
    cmd = [
        "ffmpeg", "-i", video_path, "-ss", str(start_sec), "-t", "10",
        "-ar", "16000", "-ac", "1", "-c:a", "pcm_s16le", audio_path,
        "-y"
    ]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # Save metadata (label)
    with open(metadata_path, "w") as f:
        f.write(f"Video ID: {video_id}\nLabel: {label}\nSplit: {split}\n")
    return f"Processed {video_id}"
# Read CSV and process videos in parallel with progress tracking
def download_vggsound():
    with open(CSV_FILE, "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        video_list = [row for row in reader]
    total_videos = len(video_list)
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust workers for speed
        future_to_video = {executor.submit(process_video, vid): vid for vid in video_list}
        # Display progress bar
        with tqdm(total=total_videos, desc="Downloading and Processing", unit="video") as pbar:
            for future in as_completed(future_to_video):
                result = future.result()
                pbar.set_postfix_str(result)
                pbar.update(1)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\n:white_check_mark: Download and processing completed in {elapsed_time:.2f} seconds (~{elapsed_time / total_videos:.2f} sec/video)")
if __name__ == "__main__":
    download_vggsound()

(199175, 5)